## Vectorization of the cleaned image.


This code perfoms skeletonization on the images and then fits bezier curves to the curves

Declaring the libraries

In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from skimage.morphology import skeletonize, disk, opening, closing, square
from skimage.filters import threshold_otsu, threshold_local
import matplotlib.pyplot as plt
import networkx as nx
import BezierFit
import ObjectProfileFitting
import cv2
%matplotlib inline
from skimage.measure import label, regionprops
import os
import cv2

import time

Code which find branches according to junctions. Branches shorter than the threshold are ignored.

The function ObjectProfileFitting is a obtained from 

@InProceedings{Bonnici2020,
  author    = {Bonnici,Alexandra and Camilleri, Kenneth P.},
  booktitle = {Proceedings of the ACM Symposium on Document Engineering 2020},
  title     = {Machine Interpretation of sketched Documents},
  year      = {2020},
  doi       = {10.1145/3395027.3419577},
  keywords  = {Over-sketching, sketch-interaction, Gabor Filter, Deep Learning},
  url       = {https:/doi.rg/10.1145/3395027.3419577},
}

In [22]:
def bezier_curves(im_skel, Maximum_Fit_Error, Tangent_Support_Region, name, prune):
    plt.imshow(np.zeros((im_skel.shape))*255,cmap='binary')
    col = ['yellow', 'blue', 'green']

    
    N = np.ones((3, 3))
    N[1,1] = 0

    Conv_Ann = cv2.filter2D(im_skel, -1, N) 

    #Junction
    junc =np.zeros(im_skel.shape)
    junc[Conv_Ann >2] = 1
    junc_connected = np.minimum(im_skel,junc)

    im_array_junc =np.asarray (np.nonzero(junc_connected >0))
    im_array_junc = im_array_junc.transpose()


    #end point
    end =np.zeros(im_skel.shape)
    end[Conv_Ann == 1 ] = 1
    end_connected = np.minimum(im_skel,end)

    im_array_end =np.asarray (np.nonzero(end_connected >0))
    im_array_end = im_array_end.transpose()


    Conv_Ann = Conv_Ann*im_skel

    Conv_Ann_copy = Conv_Ann


    

    for i in range(0,len(im_array_junc),1):
        a,b = im_array_junc[i]

        end = 0

        for j in range(0,(Conv_Ann_copy[a,b]),1):
            L = []  
            L[len(L):] = [(a,b)]
            n_a,n_b = a,b
            Conv_Ann[a,b] = 0

            while True:
                if (Conv_Ann[n_a+1,n_b] == 2 ):
                    Conv_Ann[n_a+1,n_b] = 0
                    L[len(L):] = [(n_a+1,n_b)]
                    n_a,n_b = n_a+1,n_b

                elif (Conv_Ann[n_a,n_b+1] == 2):
                    Conv_Ann[n_a,n_b+1] = 0
                    L[len(L):] = [(n_a,n_b+1)]
                    n_a,n_b = n_a,n_b+1

                elif (Conv_Ann[n_a-1,n_b] == 2):
                    Conv_Ann[n_a-1,n_b] = 0
                    L[len(L):] = [(n_a-1,n_b)]
                    n_a,n_b = n_a-1,n_b

                elif (Conv_Ann[n_a,n_b-1] == 2):
                    Conv_Ann[n_a,n_b-1] = 0
                    L[len(L):] = [(n_a,n_b-1)]
                    n_a,n_b = n_a,n_b-1

                elif (Conv_Ann[n_a+1,n_b+1] == 2):
                    Conv_Ann[n_a+1,n_b+1] = 0
                    L[len(L):] = [(n_a+1,n_b+1)]
                    n_a,n_b = n_a+1,n_b+1

                elif (Conv_Ann[n_a+1,n_b-1] == 2):
                    Conv_Ann[n_a+1,n_b-1] = 0
                    L[len(L):] = [(n_a+1,n_b-1)]
                    n_a,n_b = n_a+1,n_b-1

                elif (Conv_Ann[n_a-1,n_b-1] == 2):
                    Conv_Ann[n_a-1,n_b-1] = 0
                    L[len(L):] = [(n_a-1,n_b-1)]
                    n_a,n_b = n_a-1,n_b-1

                elif (Conv_Ann[n_a-1,n_b+1] == 2):
                    Conv_Ann[n_a-1,n_b+1] = 0
                    L[len(L):] = [(n_a-1,n_b+1)]
                    n_a,n_b = n_a-1,n_b+1

                elif (Conv_Ann[n_a+1,n_b] == 1):
                    Conv_Ann[n_a+1,n_b] = 0
                    L[len(L):] = [(n_a+1,n_b)]
                    n_a,n_b = a,b
                    end = 1
                    break

                elif (Conv_Ann[n_a,n_b+1] == 1):
                    Conv_Ann[n_a,n_b+1] = 0
                    L[len(L):] = [(n_a,n_b+1)]
                    n_a,n_b = a,b
                    end = 1
                    break

                elif (Conv_Ann[n_a-1,n_b] == 1):
                    Conv_Ann[n_a-1,n_b] = 0
                    L[len(L):] = [(n_a-1,n_b)]
                    n_a,n_b = a,b
                    end = 1
                    break

                elif (Conv_Ann[n_a,n_b-1] == 1):
                    Conv_Ann[n_a,n_b-1] = 0
                    L[len(L):] = [(n_a,n_b-1)]
                    n_a,n_b = a,b
                    end = 1
                    break

                elif (Conv_Ann[n_a+1,n_b+1] == 1):
                    Conv_Ann[n_a+1,n_b+1] = 0
                    L[len(L):] = [(n_a+1,n_b+1)]
                    n_a,n_b = a,b
                    end = 1
                    break

                elif (Conv_Ann[n_a+1,n_b-1] == 1):
                    Conv_Ann[n_a+1,n_b-1] = 0
                    L[len(L):] = [(n_a+1,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b-1] == 1):
                    Conv_Ann[n_a-1,n_b-1] = 0
                    L[len(L):] = [(n_a-1,n_b-1)]
                    n_a,n_b = a,b
                    end = 1
                    break

                elif (Conv_Ann[n_a-1,n_b+1] == 1):
                    Conv_Ann[n_a-1,n_b+1] = 0
                    L[len(L):] = [(n_a-1,n_b+1)]
                    n_a,n_b = a,b
                    end = 1
                    break

                elif (Conv_Ann[n_a+1,n_b] > 2):
                    L[len(L):] = [(n_a+1,n_b)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a,n_b+1] > 2):
                    L[len(L):] = [(n_a,n_b+1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b] > 2):
                    L[len(L):] = [(n_a-1,n_b)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a,n_b-1] > 2):
                    L[len(L):] = [(n_a,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a+1,n_b+1] > 2):
                    L[len(L):] = [(n_a+1,n_b+1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a+1,n_b-1] > 2):
                    L[len(L):] = [(n_a+1,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b-1] > 2):
                    #Conv_Ann[n_a-1,n_b-1] = 0
                    L[len(L):] = [(n_a-1,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b+1] > 2):
                    L[len(L):] = [(n_a-1,n_b+1)]
                    n_a,n_b = a,b
                    break
                else :
                    n_a,n_b = a,b    
                    break

            L = np.asarray(L)
            Q = L[0, :].reshape(1, 2)
            im_array_plot= np.r_[L]
            
            if (end ==0  ):

                Object_Profile = ObjectProfileFitting.object_profile_fitting(im_array_plot, Maximum_Fit_Error, Tangent_Support_Region)
                cy = (Object_Profile[:,0])
                cx = (Object_Profile[:,1])

                plt.plot(cx,cy,'-', color ='black', linewidth=1, solid_capstyle='round')
            else:
                if (len(L) >prune ):
                    Object_Profile = ObjectProfileFitting.object_profile_fitting(im_array_plot, Maximum_Fit_Error, Tangent_Support_Region)
                    cy = (Object_Profile[:,0])
                    cx = (Object_Profile[:,1])

                    plt.plot(cx,cy,'-', color ='black', linewidth=1, solid_capstyle='round')

    for i in range(0,len(im_array_end),1):
        a,b = im_array_end[i]

        for j in range(0,(1),1):
            L = []  
            L[len(L):] = [(a,b)]
            n_a,n_b = a,b
            Conv_Ann[a,b] = 0
            
            while True:

                if (Conv_Ann[n_a+1,n_b] == 2 ): 
                    Conv_Ann[n_a+1,n_b] = 0
                    L[len(L):] = [(n_a+1,n_b)]
                    n_a,n_b = n_a+1,n_b

                elif (Conv_Ann[n_a,n_b+1] == 2):
                    Conv_Ann[n_a,n_b+1] = 0
                    L[len(L):] = [(n_a,n_b+1)]
                    n_a,n_b = n_a,n_b+1

                elif (Conv_Ann[n_a-1,n_b] == 2):
                    Conv_Ann[n_a-1,n_b] = 0
                    L[len(L):] = [(n_a-1,n_b)]
                    n_a,n_b = n_a-1,n_b

                elif (Conv_Ann[n_a,n_b-1] == 2):
                    Conv_Ann[n_a,n_b-1] = 0
                    L[len(L):] = [(n_a,n_b-1)]
                    n_a,n_b = n_a,n_b-1

                elif (Conv_Ann[n_a+1,n_b+1] == 2):
                    Conv_Ann[n_a+1,n_b+1] = 0
                    L[len(L):] = [(n_a+1,n_b+1)]
                    n_a,n_b = n_a+1,n_b+1

                elif (Conv_Ann[n_a+1,n_b-1] == 2):
                    Conv_Ann[n_a+1,n_b-1] = 0
                    L[len(L):] = [(n_a+1,n_b-1)]
                    n_a,n_b = n_a+1,n_b-1

                elif (Conv_Ann[n_a-1,n_b-1] == 2):
                    Conv_Ann[n_a-1,n_b-1] = 0
                    L[len(L):] = [(n_a-1,n_b-1)]
                    n_a,n_b = n_a-1,n_b-1

                elif (Conv_Ann[n_a-1,n_b+1] == 2):
                    Conv_Ann[n_a-1,n_b+1] = 0
                    L[len(L):] = [(n_a-1,n_b+1)]
                    n_a,n_b = n_a-1,n_b+1

                elif (Conv_Ann[n_a+1,n_b] == 1):
                    Conv_Ann[n_a+1,n_b] = 0
                    L[len(L):] = [(n_a+1,n_b)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a,n_b+1] == 1):
                    Conv_Ann[n_a,n_b+1] = 0
                    L[len(L):] = [(n_a,n_b+1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b] == 1):
                    Conv_Ann[n_a-1,n_b] = 0
                    L[len(L):] = [(n_a-1,n_b)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a,n_b-1] == 1):
                    Conv_Ann[n_a,n_b-1] = 0
                    L[len(L):] = [(n_a,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a+1,n_b+1] == 1):
                    Conv_Ann[n_a+1,n_b+1] = 0
                    L[len(L):] = [(n_a+1,n_b+1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a+1,n_b-1] == 1):
                    Conv_Ann[n_a+1,n_b-1] = 0
                    L[len(L):] = [(n_a+1,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b-1] == 1):
                    Conv_Ann[n_a-1,n_b-1] = 0
                    L[len(L):] = [(n_a-1,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b+1] == 1):
                    Conv_Ann[n_a-1,n_b+1] = 0
                    L[len(L):] = [(n_a-1,n_b+1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a+1,n_b] > 2):
                    L[len(L):] = [(n_a+1,n_b)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a,n_b+1] > 2):
                    L[len(L):] = [(n_a,n_b+1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b] > 2):
                    L[len(L):] = [(n_a-1,n_b)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a,n_b-1] > 2):
                    L[len(L):] = [(n_a,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a+1,n_b+1] > 2):
                    L[len(L):] = [(n_a+1,n_b+1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a+1,n_b-1] > 2):
                    L[len(L):] = [(n_a+1,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b-1] > 2):
                    L[len(L):] = [(n_a-1,n_b-1)]
                    n_a,n_b = a,b
                    break

                elif (Conv_Ann[n_a-1,n_b+1] > 2): 
                    L[len(L):] = [(n_a-1,n_b+1)]
                    n_a,n_b = a,b
                    break
                else :
                    n_a,n_b = a,b    
                    break

            L = np.asarray(L)
            Q = L[0, :].reshape(1, 2)
            im_array_plot= np.r_[L]

            if (len(L) > 0 ):

                Object_Profile = ObjectProfileFitting.object_profile_fitting(im_array_plot, Maximum_Fit_Error, Tangent_Support_Region)
                cy = (Object_Profile[:,0])
                cx = (Object_Profile[:,1])

                plt.plot(cx,cy,'-', color ='black', linewidth=1, solid_capstyle='round')
                  
    print(np.sum(Conv_Ann))
    while (np.sum(Conv_Ann)> 0):
        junc =np.zeros(im_skel.shape)


        con =np.zeros(im_skel.shape)
        con[Conv_Ann == 2] = 1
        con = np.minimum(im_skel,con)

        im_array_con =np.asarray (np.nonzero(con>0))
        im_array_con = im_array_con.transpose()


        a,b = im_array_con[0]
        L = []  
        L[len(L):] = [(a,b)]

        n_a,n_b = a,b
        Conv_Ann[a,b] = 0

        print(Conv_Ann[a-1:a+2,b-1:b+2])
        while True:

            if (Conv_Ann[n_a+1,n_b] == 2 ): 
                Conv_Ann[n_a+1,n_b] = 0
                L[len(L):] = [(n_a+1,n_b)]
                n_a,n_b = n_a+1,n_b

            elif (Conv_Ann[n_a,n_b+1] == 2):
                Conv_Ann[n_a,n_b+1] = 0
                L[len(L):] = [(n_a,n_b+1)]
                n_a,n_b = n_a,n_b+1

            elif (Conv_Ann[n_a-1,n_b] == 2):
                Conv_Ann[n_a-1,n_b] = 0
                L[len(L):] = [(n_a-1,n_b)]
                n_a,n_b = n_a-1,n_b

            elif (Conv_Ann[n_a,n_b-1] == 2):
                Conv_Ann[n_a,n_b-1] = 0
                L[len(L):] = [(n_a,n_b-1)]
                n_a,n_b = n_a,n_b-1

            elif (Conv_Ann[n_a+1,n_b+1] == 2):
                Conv_Ann[n_a+1,n_b+1] = 0
                L[len(L):] = [(n_a+1,n_b+1)]
                n_a,n_b = n_a+1,n_b+1

            elif (Conv_Ann[n_a+1,n_b-1] == 2):
                Conv_Ann[n_a+1,n_b-1] = 0
                L[len(L):] = [(n_a+1,n_b-1)]
                n_a,n_b = n_a+1,n_b-1

            elif (Conv_Ann[n_a-1,n_b-1] == 2):
                Conv_Ann[n_a-1,n_b-1] = 0
                L[len(L):] = [(n_a-1,n_b-1)]
                n_a,n_b = n_a-1,n_b-1

            elif (Conv_Ann[n_a-1,n_b+1] == 2):
                Conv_Ann[n_a-1,n_b+1] = 0
                L[len(L):] = [(n_a-1,n_b+1)]
                n_a,n_b = n_a-1,n_b+1

            else :
                n_a,n_b = a,b    
                break

        L = np.asarray(L)
        Q = L[0, :].reshape(1, 2)
        im_array_plot= np.r_[L]

        if (len(L) > 0 ):

            Object_Profile = ObjectProfileFitting.object_profile_fitting(im_array_plot, Maximum_Fit_Error, Tangent_Support_Region)
            cy = (Object_Profile[:,0])
            cx = (Object_Profile[:,1])

            plt.plot(cx,cy,'-', color ='black', linewidth=1, solid_capstyle='round')
 
    
    end = time.time()
    
    plt.axis('off')
    plt.savefig(name, format='pdf',bbox_inches='tight')

    return end , im_array_junc.shape, im_array_end.shape

User specifictions for closeing morphology,pruning and Bezier curves.

In [14]:
#bezier curve setings, it is not sugested that you change these, but you could
Maximum_Fit_Error = 5
Tangent_Support_Region = 4
#set to True if you think a closing operation is necessary, set to False if you don't
Close = False
#maximum length of the branches that will be ignored
prune = 40

In [4]:
#use this code to create the output directory
#os.mkdir("output") 

Loop to vectorize all the images in a file. the images in he file need to be named named as numbers, ex. '1.jpg','2.jpg'ect. The variable name_i needs to be the adress of these inputs. The variable name needs to be the address of the output. The output is in pdf format.



In [23]:
time_avg = 0 
for k in range(1,20):
    name_i = '{}.jpg'.format(k)
    img = cv2.imread(name_i)
    im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    n,m = im_gray.shape
    
    
    start = time.time()
    im_gray  = cv2.resize(im_gray ,(int(m*2),int(n*2)))
    #thresh = threshold_otsu(im_gray)
    thresh = 250

    # use the threshold to binarise theimage
    kernel = np.ones((3,3),dtype =np.uint8)


    im_bin_inv = im_gray > thresh

    a,b = im_bin_inv.shape
    print(a,b)

    im_bin_inv_temp = np.ones((a+4,b+4), dtype = 'uint8')

    
    im_bin_inv_temp[2:a+2,2:b+2] = im_bin_inv
    #m_bin_inv_temp= im_bin_inv

    im_bin = np.invert(im_bin_inv_temp)
    if (Close == True):
        im_bin = cv2.morphologyEx(np.invert(im_bin_inv_temp ),cv2.MORPH_CLOSE,kernel,iterations = 5)
    else:
        im_bin = np.invert(im_bin_inv_temp )
        
    im_bin=im_bin//im_bin.max()

    im_skel = skeletonize(im_bin, method='zhang')
    im_skel= np.uint8(im_skel*1)
    
    
    name = '_40_close.pdf'.format(k)
    end, junc, endpt=bezier_curves(im_skel,Maximum_Fit_Error,Maximum_Fit_Error, name, prune)
    
    pixel_am= n*m
    timeperpixel = (end - start)/pixel_am
    
    print(timeperpixel)
    
    time_avg = time_avg + (end - start)
    
    timeperjunc = (end - start)/(junc[0]+endpt[0])
    plt.clf()
    print(timeperjunc)
    print(end-start)
print('Avg:',time_avg, 'Per_jun:', timeperjunc )

2068 2842
548
[[0 0 0]
 [0 0 2]
 [2 0 0]]
1.5907178866968673e-05
0.06956143038613456
23.37264060974121
Avg: 23.37264060974121 Per_jun: 0.06956143038613456


<Figure size 432x288 with 0 Axes>